
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



In [10]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging

import pandas as pd
import ExMAS


## Pricing for Delft, Netherlands

In [12]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 0.1 # 6 minutes hour of simulation
params.nP = 10# reuqests (and passengers)
params.nV = 10# vehicles

## Parameters for ExMAS

In [13]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.3
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_pax' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True

inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 


06-02-23 12:15:05-INFO-Initializing pairwise trip shareability between 10 and 10 trips.
06-02-23 12:15:05-INFO-creating combinations
06-02-23 12:15:05-INFO-90	 nR*(nR-1)
06-02-23 12:15:05-INFO-Reduction of feasible pairs by 96.67%
06-02-23 12:15:05-INFO-Degree 2 	Completed
06-02-23 12:15:05-INFO-trips to extend at degree 2 : 8
06-02-23 12:15:05-INFO-At degree 2 feasible extensions found out of 0 searched
06-02-23 12:15:05-INFO-Degree 3 	Completed
06-02-23 12:15:05-INFO-No more trips to exted at degree 3


In [14]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

06-02-23 12:15:05-INFO-Matching 10 trips to 18 rides in order to min u_pax
06-02-23 12:15:05-INFO-Problem solution: Optimal. 
Total costs for single trips:             53 
reduced by matching to:                   50


In [15]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],6.66600,360,1,[6.6659999999999995],"[0, 360]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1402644600, 5124546092]","[None, 0, 0]",node time req_id od 0 ...,360,3.60,5.40600,1.08120,4.32480
1,[1],7.78850,421,1,[7.788500000000001],"[14, 421]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44813589, 1999885354]","[None, 1, 1]",node time req_id od 0 ...,421,4.21,6.31500,1.26300,5.05200
2,[2],4.05150,219,1,[4.0515],"[49, 219]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3790294279, 44808578]","[None, 2, 2]",node time req_id od 0 ...,219,2.19,3.28500,0.65700,2.62800
3,[3],9.59200,518,1,[9.592],"[89, 518]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44908875, 1679761049]","[None, 3, 3]",node time req_id od 0 ...,518,5.18,7.77900,1.55580,6.22320
4,[4],8.83050,477,1,[8.830499999999999],"[146, 477]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 310785561, 1552651574]","[None, 4, 4]",node time req_id od 0 ...,477,4.77,7.16100,1.43220,5.72880
5,[5],3.09100,167,1,[3.091],"[215, 167]",[5],[5],1,5,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",1,"[None, 1402771630, 894421661]","[None, 5, 5]",node time req_id od 0 ...,167,1.67,2.50650,0.50130,2.00520
6,[6],3.06600,165,1,[3.0660000000000003],"[230, 165]",[6],[6],1,6,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",1,"[None, 44759522, 2321923795]","[None, 6, 6]",node time req_id od 0 ...,165,1.65,2.48850,0.49770,1.99080
7,[7],1.85600,100,1,[1.856],"[294, 100]",[7],[7],1,7,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[None, 626009978, 1679761156]","[None, 7, 7]",node time req_id od 0 ...,100,1.00,1.50600,0.30120,1.20480
8,[8],4.20400,227,1,[4.204],"[322, 227]",[8],[8],1,8,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0,"[None, 1846605952, 1571093185]","[None, 8, 8]",node time req_id od 0 ...,227,2.27,3.40950,0.68190,2.72760
9,[9],4.57100,247,1,[4.571],"[325, 247]",[9],[9],1,9,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1552651150, 4846026465]","[None, 9, 9]",node time req_id od 0 ...,247,2.47,3.70650,0.74130,2.96520


## Simulation with MaaSSim 

In [16]:
sim = simulate(params = params, inData = inData, logger_level = logging.WARNING) # simulate

06-02-23 12:15:08-INFO-Matching 10 trips to 18 rides in order to min u_pax
06-02-23 12:15:08-INFO-Problem solution: Optimal. 
Total costs for single trips:             53 
reduced by matching to:                   50
06-02-23 12:15:08-WARNING-Setting up 0.1h simulation at 2023-02-06 12:12:20 for 10 vehicles and 10 passengers in Delft, Netherlands
06-02-23 12:15:09-CRITICAL-this is request 0 with [0, 10, 11, 17] available rides.
06-02-23 12:15:09-CRITICAL-ride 0 available [1]
06-02-23 12:15:09-CRITICAL-ride 10 available [1, 0]
06-02-23 12:15:09-CRITICAL-ride 11 available [1, 0]
06-02-23 12:15:09-CRITICAL-ride 17 available [0, 1]
06-02-23 12:15:09-CRITICAL-this is reuqest 0 with [0, 10, 11, 17] still available rides.
06-02-23 12:15:09-CRITICAL-vehicle 1 has 4 choices
06-02-23 12:15:09-CRITICAL-vehicle 1 has chosen to serve request 0 with a ride 17 of degree 2, with travellers [4, 0].
06-02-23 12:15:09-CRITICAL-this is request 1 with [1, 12, 13] available rides.
06-02-23 12:15:09-CRITICAL

In [18]:
inData.vehicles.head(10)

,pos,event,shift_start,shift_end,platform,expected_income
veh,,,,,,
1,1552651574,driverEvent.ENDS_SHIFT,0,86400,0,NaN
2,1679761156,driverEvent.ENDS_SHIFT,0,86400,0,NaN
3,44829675,driverEvent.ENDS_SHIFT,0,86400,0,NaN
4,1569647751,driverEvent.ENDS_SHIFT,0,86400,0,NaN
5,1679761049,driverEvent.ENDS_SHIFT,0,86400,0,NaN
6,44855661,driverEvent.IS_ACCEPTED_BY_TRAVELLER,0,86400,0,NaN
7,1570983098,driverEvent.ENDS_SHIFT,0,86400,0,NaN
8,1999885354,driverEvent.ENDS_SHIFT,0,86400,0,NaN
9,2321923795,driverEvent.ENDS_SHIFT,0,86400,0,NaN


In [19]:
pd.DataFrame(sim.vehs[1].myrides)


,veh,pos,t,event,paxes
0,1,1.608998e+09,0.0,STARTS_DAY,[]
1,1,1.608998e+09,0.0,OPENS_APP,[]
2,1,1.608998e+09,0.0,RECEIVES_REQUEST,[]
3,1,1.608998e+09,15.0,ACCEPTS_REQUEST,[]
4,1,1.608998e+09,35.0,IS_ACCEPTED_BY_TRAVELLER,[]
5,1,1.608998e+09,181.0,IS_ACCEPTED_BY_TRAVELLER,[]
6,1,1.402645e+09,569.0,ARRIVES_AT_PICKUP,[]
7,1,1.402645e+09,569.0,MEETS_TRAVELLER_AT_PICKUP,[]
8,1,1.402645e+09,599.0,DEPARTS_FROM_PICKUP,[0]
9,1,3.107856e+08,755.0,ARRIVES_AT_PICKUP,[0]


In [22]:
sim.res[0].pax_exp


,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE,TRAVEL,WAIT,OPERATIONS
pax,,,,,,,,,,,,,,,,,
0,15.0,568.0,20.0,30.0,423.0,111.0,0.0,10.0,0,0,0,0,0,0,568.0,534.0,55.0
1,15.0,567.0,0.0,30.0,214.0,20.0,14.0,10.0,0,0,0,0,0,0,567.0,234.0,55.0
2,15.0,276.0,35.0,30.0,25.0,14.0,20.0,10.0,0,0,0,0,0,0,276.0,39.0,55.0
3,15.0,831.0,20.0,30.0,87.0,81.0,89.0,10.0,0,0,0,0,0,0,831.0,168.0,55.0
4,15.0,558.0,35.0,30.0,574.0,0.0,131.0,10.0,0,0,0,0,0,0,558.0,574.0,55.0
5,15.0,167.0,20.0,30.0,138.0,0.0,215.0,10.0,0,0,0,0,0,0,167.0,138.0,55.0
6,15.0,165.0,20.0,30.0,70.0,0.0,230.0,10.0,0,0,0,0,0,0,165.0,70.0,55.0
7,15.0,100.0,20.0,30.0,173.0,0.0,294.0,10.0,0,0,0,0,0,0,100.0,173.0,55.0
8,15.0,227.0,35.0,30.0,188.0,89.0,218.0,10.0,0,0,0,0,0,0,227.0,277.0,55.0


In [24]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM,REVENUE
veh,,,,,,,,,,,,,,,,,,,,,
1,2.0,0.0,False,0,0,0.0,15.0,0,166.0,0,...,0,0,6216.0,558.0,0,544.0,241.0,4873.0,8.378,5.72880
2,1.0,0.0,False,0,0,294.0,15.0,0,20.0,0,...,0,0,6927.0,100.0,0,173.0,65.0,6589.0,1.004,1.20480
3,0.0,0.0,False,0,0,146.0,0.0,0,0.0,0,...,0,0,7413.0,0.0,0,0.0,0.0,7413.0,0.000,0.00000
4,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0,0,7559.0,0.0,0,0.0,0.0,7559.0,0.000,0.00000
5,2.0,0.0,False,0,0,89.0,15.0,0,136.0,0,...,0,0,6406.0,578.0,0,275.0,211.0,5342.0,7.459,6.26556
6,0.0,0.0,False,0,0,325.0,15.0,0,20.0,0,...,0,0,0.0,0.0,0,0.0,35.0,-35.0,2.471,0.00000
7,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0,0,7559.0,0.0,0,0.0,0.0,7559.0,0.000,0.00000
8,2.0,0.0,False,0,0,14.0,15.0,0,55.0,0,...,0,0,6699.0,567.0,0,149.0,130.0,5853.0,6.400,5.37600
9,1.0,0.0,False,0,0,230.0,15.0,0,20.0,0,...,0,0,7029.0,165.0,0,70.0,65.0,6729.0,1.659,1.99080


In [25]:
sim.res[0].all_kpi

22.571159999999995

In [26]:
from MaaSSim.driver import VehicleAgent, driverEvent
import pandas as pd
pd.DataFrame([[s,s.name,s.value] for s in driverEvent], 
             columns = ['status','name','value'])

,status,name,value
0,driverEvent.STARTS_DAY,STARTS_DAY,0
1,driverEvent.OPENS_APP,OPENS_APP,1
2,driverEvent.RECEIVES_REQUEST,RECEIVES_REQUEST,2
3,driverEvent.ACCEPTS_REQUEST,ACCEPTS_REQUEST,3
4,driverEvent.REJECTS_REQUEST,REJECTS_REQUEST,4
5,driverEvent.IS_ACCEPTED_BY_TRAVELLER,IS_ACCEPTED_BY_TRAVELLER,5
6,driverEvent.IS_REJECTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,6
7,driverEvent.ARRIVES_AT_PICKUP,ARRIVES_AT_PICKUP,7
8,driverEvent.MEETS_TRAVELLER_AT_PICKUP,MEETS_TRAVELLER_AT_PICKUP,8
9,driverEvent.DEPARTS_FROM_PICKUP,DEPARTS_FROM_PICKUP,9
